<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Get-files" data-toc-modified-id="Get-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get files</a></span></li><li><span><a href="#Extracting-features" data-toc-modified-id="Extracting-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting features</a></span></li><li><span><a href="#Export-DataFrame-to-.CSV" data-toc-modified-id="Export-DataFrame-to-.CSV-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export DataFrame to .CSV</a></span></li></ul></div>

# Libraries

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
# For audio playing
import IPython.display as ipd
# For visualizations
import librosa.display
from math import ceil

# Get files 

**To maximize the results obtained from the data and ensure that all samples have the same duration we will use the following function.**

1. Navigate through directories to get files.</li>
2. For each file we get several fragments that overlap the previous and following fragment.</li>


In [2]:
directory = "data"
def gen_audio(window,overlap=0,path=directory):
    for genre in os.listdir(path):
        if os.path.isdir(f"{path}/{genre}"):
            folder=f"{path}/{genre}"
            for song in os.listdir(folder):
                song_path = f"{folder}/{song}"
                if song == ".DS_Store":
                    continue
                y, sr = librosa.load(song_path)
                duration = True
                i=0
                while duration:
                    #print(song_path)
                    song_points = len(y)
                    window_points = window*sr
                    start = int(i*window_points - overlap*window_points)
                    if start<0:
                        start=0
                    end = start + window_points
                    #print(start,end)
                    sample=y[start:end]
                    if end >= song_points:
                        #print("-")
                        sample = y[-window_points:]
                        duration = False
                    i += 1
                    yield genre,sample,sr

In [3]:
audio_generator = gen_audio(15,0.25)

In [4]:
y_df = pd.DataFrame()
X = pd.DataFrame()

# Extracting features

**The features that are going to be used are:**
- Zero crossings
- Spectral centroid
- Spectral band_width
- Spectral rolloff
- Chroma stft
- Mel Frequency Cepstral Coeﬃcients (mfcc)
- Tonal centroid
- Onset_strength
- PLP
- Beats per min

In [5]:
%%time
i=1
for label, sample, sr in audio_generator:
    y = sample
    zero_crossing = librosa.zero_crossings(y=y, pad=False)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_band_width = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    tonal_centroid = librosa.feature.tonnetz(y=y, sr=sr)
    onset_strength = librosa.onset.onset_strength(y=y, sr=sr)
    plp = librosa.beat.plp(onset_envelope=onset_strength, sr=sr)
    y_df = y_df.append({
        "label":label,
        
    }, ignore_index=True)
    x = np.hstack([[sum(zero_crossing)],spectral_centroid[0],spectral_band_width[0],spectral_rolloff[0],
                  np.mean(chroma_stft,axis=1)])
    X = X.append(pd.Series(x),ignore_index=True)
    print(i,end="\r")
    i+=1
    break

CPU times: user 12.4 s, sys: 908 ms, total: 13.3 s
Wall time: 11.6 s


In [6]:
len(x)

1951

In [7]:
len(mfcc)

20

In [46]:
X["label"] = y_df

In [47]:
X.to_pickle("test.pkl")

In [13]:
df.head(1)

,beats,chroma_stft,label,mfcc,onset_strength,plp,sample,spectral_band_width,spectral_centroid,spectral_rolloff,sr,tonal_centroid,zero_crossings
0,189.402167,"[0.45334435, 0.39936936, 0.44807765, 0.5161125...",Supernatural,"[-242.92453, 136.92271, -24.217505, 26.947405,...",1.447724,0.273632,"[3.5482785e-08, 1.0161084e-08, -9.557592e-09, ...",1507.378098,1522.945223,2815.416309,22050.0,"[0.45334435, 0.39936936, 0.44807765, 0.5161125...",29991.0


In [14]:
df[['chroma_stft1','chroma_stft2','chroma_stft3','chroma_stft4','chroma_stft5','chroma_stft6','chroma_stft7','chroma_stft8','chroma_stft9','chroma_stft10','chroma_stft11','chroma_stft12']] = pd.DataFrame(df.chroma_stft.tolist(), index= df.index)
df = df.drop(['chroma_stft'], axis=1)

In [15]:
df[['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20']] = pd.DataFrame(df.mfcc.tolist(), index= df.index)
df = df.drop(['mfcc'], axis=1)

In [20]:
df['tonal_centroid'].iloc[0]

[0.45334435,
 0.39936936,
 0.44807765,
 0.5161125,
 0.6047815,
 0.6462182,
 0.5601424,
 0.42375207,
 0.4746412,
 0.29928055,
 0.2602959,
 0.29277796]

In [21]:
df[['tc1','tc2','tc3','tc4','tc5','tc6','tc7','tc8','tc9','tc10','tc11','tc12']] = pd.DataFrame(df.tonal_centroid.tolist(), index= df.index)
df = df.drop(['tonal_centroid'], axis=1)

In [25]:
df.head()

,beats,label,onset_strength,plp,sample,spectral_band_width,spectral_centroid,spectral_rolloff,sr,zero_crossings,...,tc3,tc4,tc5,tc6,tc7,tc8,tc9,tc10,tc11,tc12
0,189.402167,Supernatural,1.447724,0.273632,"[3.5482785e-08, 1.0161084e-08, -9.557592e-09, ...",1507.378098,1522.945223,2815.416309,22050.0,29991.0,...,0.448078,0.516113,0.604782,0.646218,0.560142,0.423752,0.474641,0.299281,0.260296,0.292778
1,185.361378,Supernatural,1.318500,0.267129,"[-0.018045332, -0.022796264, -0.022938928, -0....",1122.701344,755.909261,1388.591603,22050.0,9221.0,...,0.514236,0.568662,0.578250,0.528015,0.594869,0.435590,0.402993,0.282640,0.320256,0.293354
2,185.282430,Supernatural,1.248585,0.278978,"[0.119748116, 0.11298294, 0.10486316, 0.095828...",1578.514212,993.390330,2033.504370,22050.0,11084.0,...,0.308524,0.431915,0.466115,0.401818,0.469616,0.466222,0.616621,0.519215,0.422129,0.354170
3,185.387694,Supernatural,1.199493,0.260926,"[0.62980396, 0.6035918, 0.5775655, 0.5571881, ...",1600.937584,950.315775,2002.187893,22050.0,10121.0,...,0.325616,0.386509,0.352333,0.331226,0.466208,0.521074,0.734965,0.552764,0.420204,0.335290
4,185.335062,Supernatural,1.212720,0.272926,"[-0.09439093, -0.06570398, -0.032624375, -0.00...",1680.028111,1023.549388,2140.703722,22050.0,10175.0,...,0.345472,0.353046,0.354804,0.330782,0.496360,0.455021,0.582542,0.469681,0.406876,0.397625


In [23]:
df.shape

(1589, 54)

# Export DataFrame to .CSV

In [24]:
df.to_csv(path_or_buf= "output/even_more_features_15s_025ov.csv",index=False)